#Mastering RAG Chatbots with DeepSeek: From Implementation to Deployment

Key Modifications from OpenAI Version:
Replace OpenAI API calls with DeepSeek API
Use DeepSeek's embedding and chat completion models
Add error handling specific to DeepSeek API

- Create a new Jupyter notebook named RAG_Chatbot_Gradio_Tutorial_DeepSeek.ipynb
- Copy the code structure I've outlined
- Replace placeholders like YOUR_DEEPSEEK_API_KEY with actual credentials
- Adjust import statements and method calls based on the actual DeepSeek API library

In [1]:
# 1. Environment Setup
# !pip install openai requests beautifulsoup4 PyPDF2 numpy gradio -q

import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import PyPDF2
# from deepseek import DeepSeek  # Hypothetical import, adjust based on actual library
from openai import OpenAI
import gradio as gr

# Set DeepSeek API Key
deepseek_client =OpenAI(api_key="sk-005d9d3de8d946ecbf9721293c44d6be", base_url="https://api.deepseek.com")


# 2. Content Extraction (Same as OpenAI version)
def scrape_website(url):
    # ... (unchanged)
    """Scrape text content from a website"""
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return ' '.join(soup.stripped_strings)
    except Exception as e:
        return f"Error scraping website: {str(e)}"

def extract_pdf_content(pdf_path):
    """Extract text from a PDF file"""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            # Corrected indentation for the return statement
            return ' '.join(page.extract_text() for page in reader.pages)
    except Exception as e:
        return f"Error processing PDF: {str(e)}"
    
def split_into_chunks(text, chunk_size=500):
    """Split text into manageable chunks"""
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# 3. Text Processing and Embedding
def generate_embeddings(text):
    """Generate embeddings using DeepSeek"""
    response = deepseek_client.embeddings.create(
        model="text-embedding-v1",  # Adjust model name as per DeepSeek's offering
        input=text
    )
    return response.embeddings[0]

def cosine_similarity(vec1, vec2):
    """Compute cosine similarity between two vectors"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 4. RAG Chatbot Core Implementation
class RAGChatbot:
    def ask(self, query):
        relevant_chunk = self.find_relevant_chunk(query)

        response = deepseek_client.chat.completions.create(
            model="deepseek-reasoner",  # Adjust model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant using context to answer questions."},
                {"role": "user", "content": f"Context: {relevant_chunk}\n\nQuery: {query}"}
            ],
            max_tokens=200
        )
        return response.choices[0].message.content


    def load_from_url(self, url):
        """Load content from a website"""
        content = scrape_website(url)
        self._process_content(content)

    def load_from_pdf(self, pdf_path):
        """Load content from a PDF"""
        content = extract_pdf_content(pdf_path)
        self._process_content(content)

    def _process_content(self, content):
        """Process content into chunks and generate embeddings"""
        chunks = split_into_chunks(content)
        self.chunks_with_embeddings = [
            {"content": chunk, "embedding": generate_embeddings(chunk)}
            for chunk in chunks
        ]

    def find_relevant_chunk(self, query):
        """Find most relevant text chunk for a query"""
        query_embedding = generate_embeddings(query)
        similarities = [
            (chunk["content"], cosine_similarity(query_embedding, chunk["embedding"]))
            for chunk in self.chunks_with_embeddings
        ]
        return max(similarities, key=lambda x: x[1])[0]
    



/home/abdoul/rag_lawyer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Gradio Web Interface

In [2]:
# --- ipython-input-7-7774a7eacb87 ---
class RAGChatbotInterface:
    def __init__(self):
        self.chatbot = RAGChatbot()
        self.chat_history = []

    def process_file(self, file):
        """Process uploaded PDF file"""
        try:
            self.chatbot.load_from_pdf(file.name)
            return "PDF successfully loaded! You can now ask questions."
        except Exception as e:
            return f"Error processing PDF: {str(e)}"

    def process_url(self, url):
        """Process website URL"""
        try:
            self.chatbot.load_from_url(url)
            return "Website content successfully loaded! You can now ask questions."
        except Exception as e:
            return f"Error processing URL: {str(e)}"

    def chat(self, message, history):
        """Process chat message and update history"""
        try:
            response = self.chatbot.ask(message)
            history.append((message, response))
            return response, history
        except Exception as e:
            error_message = f"Error generating response: {str(e)}"
            history.append((message, error_message))
            return error_message, history

    def launch_interface(self, share=True):
        """Create and launch Gradio interface"""
        with gr.Blocks(title="RAG Chatbot") as interface:
            gr.Markdown("# 📚 RAG Chatbot: Learn from Any Document")

            with gr.Tab("PDF Input"):
                # Change 'type' to 'filepath' to get the file path
                pdf_upload = gr.File(label="Upload PDF", type="filepath", file_types=[".pdf"])
                pdf_status = gr.Textbox(label="PDF Status", interactive=False)
                pdf_upload.upload(fn=self.process_file, inputs=[pdf_upload], outputs=[pdf_status])

            with gr.Tab("URL Input"):
                url_input = gr.Textbox(label="Enter Website URL", placeholder="https://example.com")
                url_status = gr.Textbox(label="URL Status", interactive=False)
                url_button = gr.Button("Load Content")
                url_button.click(fn=self.process_url, inputs=[url_input], outputs=[url_status])

            chatbot = gr.Chatbot(label="Chat with Your Document", height=400)
            msg = gr.Textbox(label="Your Question", placeholder="Ask a question about the document...")
            clear = gr.Button("Clear Chat")

            msg.submit(fn=self.chat, inputs=[msg, chatbot], outputs=[msg, chatbot])
            clear.click(lambda: None, None, chatbot, queue=False)

        interface.launch(share=share)

## 6. Launching the RAG Chatbot Interface

In [3]:
# Create and launch the interface
rag_interface = RAGChatbotInterface()
rag_interface.launch_interface(share=True)

/tmp/ipykernel_19798/4037565904.py:51: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat with Your Document", height=400)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/04/23 23:25:27 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


## Learning Outcomes

By completing this tutorial, you will have learned:

1. **Content Extraction Techniques**
   - Web scraping with BeautifulSoup
   - PDF text extraction

2. **Text Processing**
   - Text chunking strategies
   - Semantic embedding generation

3. **RAG Architecture**
   - Context retrieval
   - Similarity-based chunk selection
   - Prompt engineering

4. **Web Interface Development**
   - Creating interactive UIs with Gradio
   - Handling file and URL inputs
   - Managing chat interactions

## Challenges and Extensions

1. Implement multi-document support
2. Add more sophisticated embedding techniques
3. Improve error handling and input validation
4. Create a more advanced prompt engineering strategy
5. Implement conversation memory

## Ethical Considerations

- Always respect copyright and terms of service
- Be mindful of privacy when processing documents
- Use AI responsibly and ethically